# Packages

In [1]:
# Data analysis and Excel reading
import pandas as pd

# Load environment variables from .env file
from dotenv import load_dotenv

# File and directory operations
import os

# Send HTTP requests (e.g., to Google Street View API)
import requests

# Image processing (OpenCV)
import cv2

# Display images and HTML in Jupyter notebooks
from IPython.display import Image, display, HTML

# Time handling and delays
import time

# Excel file manipulation and styling
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# Checking Street View Images

Check which street segments have Google Street View images. For each segment, we use the first point coordinates to query the Street View Metadata API and see if imagery is available.

In [2]:
# Authenticate requests to the Google API

load_dotenv("api_key.env")
api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
# Load Excel file with coordinates

excel_file = "coordinates.xlsx"
df = pd.read_excel(excel_file)

In [4]:
# Load the existing Excel file
df = pd.read_excel("coordinates.xlsx")

# Function to check Street View availability using the Metadata API
def get_streetview_status(lat, lon, api_key):
    url = (f"https://maps.googleapis.com/maps/api/streetview/metadata?"
           f"location={lat},{lon}&key={api_key}")
    try:
        r = requests.get(url)
        data = r.json()
        if data.get("status") == "OK":
            return "Available"
        else:
            return "Not Available"
    except:
        return "Error"

# Create or update the column for Street View status only for points == 1
# and if the status is not already filled
if "street_view_status" not in df.columns:
    df["street_view_status"] = None

for idx, row in df.iterrows():
    if row["points"] == 1 and pd.isna(row["street_view_status"]):
        df.at[idx, "street_view_status"] = get_streetview_status(row["lat"], row["lon"], api_key)

# Save back to the same Excel file
df.to_excel("coordinates.xlsx", index=False)
print("Process completed.")

Process completed.


# Get image date via the metadata API

An automated extraction of image dates from Google Street View was performed using the Street View Metadata API, based on geographic coordinates retrieved from an Excel file. For each point, the date of the most recent available image was recorded.

In [ ]:
# Function to get metadata date
def get_metadata(lat, lon):
    url = "https://maps.googleapis.com/maps/api/streetview/metadata"
    params = {"location": f"{lat},{lon}",
              "key": api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data.get("date", "Unknown")
    else:
        return "Error"

# Load your original coordinates Excel
excel_file = "coordinates.xlsx"
df = pd.read_excel(excel_file)

# Create 'date' column if it does not exist
if "date" not in df.columns:
    df["date"] = None

# Update only the date column
for idx, row in df.iterrows():

    lat = row["lat"]
    lon = row["lon"]

    # Skip if date already exists and is valid
    if pd.notna(row["date"]) and row["date"] not in ["Unknown", "Error"]:
        continue

    date = get_metadata(lat, lon)

    df.at[idx, "date"] = date

# Save updated data
df.to_excel(excel_file, index=False)

# Reopen workbook to apply cell alignment
wb = load_workbook(excel_file)
ws = wb.active

alignment_center = Alignment(horizontal="center", vertical="center")

# Center all cells
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = alignment_center

wb.save(excel_file)

print("Process completed.")

# Download Street View images

Google Street View images were automatically downloaded using the Street View Static API for multiple georeferenced locations, based on start and end coordinates extracted from an Excel file. Each street segment was sampled starting from the initial point, with additional points collected every 25 meters, always including the endpoint. To avoid clustering points near the end of the segment, the penultimate sampling point was excluded if it fell within 12.5 m (half the sampling interval) of the endpoint.

Images were captured in four cardinal directions—North, East, South, and West—covering the full surroundings of each location. The images were obtained with a resolution of 640x640 pixels, a field of view of 90 degrees, and a fixed pitch angle of zero (horizontal view at street level).

All images were stored locally in folders organized by location ID.

## Parameters

In [ ]:
# Image size (width x height)
size = "640x640"

# Camera pitch (vertical angle)
pitch = 0

# Field of view (horizontal angle)
fov = 90

# Headings for cardinal directions: North, East, South, West
headings = [0, 90, 180, 270]

## Download function

In [6]:
# Function to download Street View images
def download_images(location_id, lat, lon, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    base_url = "https://maps.googleapis.com/maps/api/streetview"
    image_paths = []
    success = False

    for heading in headings:
        filename = os.path.join(output_folder, f"{location_id}_h{heading}.jpg")

        if not os.path.exists(filename):
            params = {"key": api_key,
                      "size": size,
                      "pitch": pitch,
                      "fov": fov,
                      "heading": heading,
                      "location": f"{lat},{lon}"}

            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                with open(filename, "wb") as f:
                    f.write(response.content)
                image_paths.append(filename)
                success = True
        else:
            image_paths.append(filename)
            success = True

    return success, sorted(list(set(image_paths)))


# Load Excel file with coordinates
excel_file = "coordinates.xlsx"
df = pd.read_excel(excel_file)

output_base_folder = "street_view_images"

# Initialize column to track download status
df["street_view_downloaded"] = False

# Prepare coordinates
coordinates = list(zip(df.index, df["id"], df["points"], df["lat"], df["lon"]))

# Target ID and city
target_id = 5103403535
target_city = "Cuiabá"

# Loop through coordinates and download images
for idx, location_id, point_num, lat, lon in coordinates:
    if location_id == target_id and df.at[idx, "city"] == target_city:

        # Folder per ID and point
        folder = os.path.join(output_base_folder, df.at[idx, "city"], str(location_id), f"point_{point_num}")

        # Download images
        success, _ = download_images(location_id=f"{location_id}_p{point_num}",
                                     lat=lat,
                                     lon=lon,
                                     output_folder=folder)

        df.at[idx, "street_view_downloaded"] = success
    else:
        df.at[idx, "street_view_downloaded"] = False

# Convert boolean to English strings "True" / "False"
df["street_view_downloaded"] = df["street_view_downloaded"].apply(
    lambda x: "True" if x else "False")

# Save updated Excel
df.to_excel("coordinates.xlsx", index=False)
print("Process completed.")

KeyboardInterrupt: 